In [ ]:
import pandas as pd
import numpy as np
import random
import math
import matplotlib.pyplot as plt
import time

UNATTENDED_CUSTOMER_COST = 10000
VEHICLES_COST = 1000
WAITING_DELAY_COST = 10


# Leemos el archivo CSV
def load_data(filename):
    df = pd.read_csv(filename)
    return df

# Cálculo de la distancia
def distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Función para calcular el coste de las rutas
def calculate_routes_cost(routes, data):
    total_cost = 0
    num_vehicles_used = 0
    clients_served = set()
    total_dist = 0
    total_wait_delay = 0

    for route in routes:
        if len(route) > 2:  # Si el vehículo atiende al menos a un cliente (además de ir y volver al almacén)
            num_vehicles_used += 1
            cost = 0
            time = 0
            for i in range(1, len(route)):
                customer1 = route[i-1]
                customer2 = route[i]

                dist = distance(data.loc[customer1, 'X'], data.loc[customer1, 'Y'],
                                data.loc[customer2, 'X'], data.loc[customer2, 'Y'])

                total_dist += dist
                time += dist
                cost += dist  # Coste por distancia recorrida

                if customer2 != 0:  # El almacén no cuenta para comprobar ventamas
                    if time < data.loc[customer2, 'READY_TIME']:
                        waiting_time = data.loc[customer2, 'READY_TIME'] - time
                        cost += waiting_time * WAITING_DELAY_COST  # Penalización por tiempo de espera
                        time = data.loc[customer2, 'READY_TIME']
                        total_wait_delay += waiting_time * WAITING_DELAY_COST

                    if time > data.loc[customer2, 'DUE_TIME']:
                        delay = time - data.loc[customer2, 'DUE_TIME']
                        cost += delay * WAITING_DELAY_COST  # Penalización por retraso
                        total_wait_delay += delay * WAITING_DELAY_COST

                    time += data.loc[customer2, 'SERVICE_TIME']
                    clients_served.add(customer2)  # Registramos el cliente atendido

            # Añadimos distancia de regreso al almacén (0,0)
            last_customer = route[-1]
            depot_dist = distance(data.loc[last_customer, 'X'], data.loc[last_customer, 'Y'], 0, 0)
            cost += depot_dist
            total_dist += depot_dist
            total_cost += cost

    # Penalización por número de vehículos empleados
    total_cost += num_vehicles_used * VEHICLES_COST

    # Penalización por clientes no atendidos
    all_clients = set(range(1, len(data)))
    unattended_clients = all_clients - clients_served
    total_cost += len(unattended_clients) * UNATTENDED_CUSTOMER_COST

    return total_cost, total_dist, total_wait_delay, num_vehicles_used, len(unattended_clients)

# Función para crear la tabla de las rutas con sus métricas
def create_routes_table(routes, data, problem, execution):
    all_routes_df = pd.DataFrame()
    for idx, route in enumerate(routes):
        route_data = []
        total_distance = 0
        time = 0
        accumulated_cost = 0
        print_route = True  # Inicializamos el coste acumulado
        for i in range(1, len(route)):
            if len(route) <= 2:
              print_route = False
              continue
            customer1 = route[i-1]
            customer2 = route[i]

            dist = distance(data.loc[customer1, 'X'], data.loc[customer1, 'Y'],
                            data.loc[customer2, 'X'], data.loc[customer2, 'Y'])

            start_time = time

            time += dist
            arrival_time = time

            # Manejo de tiempos de espera
            if arrival_time < data.loc[customer2, 'READY_TIME']:
                waiting_time = data.loc[customer2, 'READY_TIME'] - arrival_time
                arrival_time = data.loc[customer2, 'READY_TIME']
            else:
                waiting_time = 0

            accumulated_cost += waiting_time * WAITING_DELAY_COST

            # Calculamos el retraso solo si el cliente no es el almacén
            if customer2 != 0 and arrival_time > data.loc[customer2, 'DUE_TIME']:
                delay = arrival_time - data.loc[customer2, 'DUE_TIME']
                accumulated_cost += delay * WAITING_DELAY_COST  # Añadir penalización por retraso
            else:
                delay = 0

            # Actualizamos el coste acumulado con la distancia
            accumulated_cost += dist
            time += data.loc[customer2, 'SERVICE_TIME']
            total_distance += dist

            route_data.append({
                'Vehicle': idx + 1,
                'From Customer': f"{customer1}: ({data.loc[customer1, 'X']},  {data.loc[customer1, 'Y']})",
                'To Customer': f"{customer2}: ({data.loc[customer2, 'X']},  {data.loc[customer2, 'Y']})",
                'Travel Starts At': round(start_time, 2),
                'Arrival Time': round(arrival_time, 2),
                'Distance': round(dist,2),
                'Ready Time': data.loc[customer2, 'READY_TIME'],
                'Due Time': data.loc[customer2, 'DUE_TIME'],
                'Service Time': data.loc[customer2, 'SERVICE_TIME'],
                'Waiting Time': round(waiting_time, 2),
                'Delay': round(delay, 2),
                'Ends Delivery At': round(arrival_time + data.loc[customer2, 'SERVICE_TIME'] + waiting_time, 2),
                'Accumulated Cost': round(accumulated_cost, 2),
                'Accumulated Distance': round(total_distance, 2) # Añadir coste acumulado
            })

        # Mostramos las tablas
        if print_route:
          route_df = pd.DataFrame(route_data)
          route_df['Execution'] = execution
          all_routes_df = pd.concat([all_routes_df, route_df])
          print(f"Vehicle {idx + 1} Route Details:")
          print(route_df)
          print("\n")

    all_routes_df.to_csv(f"ACO_{problem}_execution_{execution}.csv", index=False)

# Función para visualizar las rutas
def plot_routes(routes, data, iteration):
    plt.figure(figsize=(10, 7))
    depot = (0, 0)
    plt.scatter(depot[0], depot[1], c='red', s=100, label='Depot')

    for idx, route in enumerate(routes):
        route_coords = [(depot[0], depot[1])] + [(data.loc[customer, 'X'], data.loc[customer, 'Y']) for customer in route[1:]] + [(depot[0], depot[1])]
        route_coords = np.array(route_coords)
        plt.plot(route_coords[:, 0], route_coords[:, 1], marker='o', label=f'Vehicle {idx+1}')

    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.title(f'Vehicle Routes - Iteration {iteration}')
    plt.legend()
    plt.show()

# Implementación del Algoritmo ACO
class ACO_VRPTW:
    def __init__(self, data, n_ants, n_best, n_iterations, alpha, beta, rho, q, n_vehicles, problem, execution):
        self.data = data
        self.n_ants = n_ants
        self.n_best = n_best
        self.n_iterations = n_iterations
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.q = q
        self.n_vehicles = n_vehicles
        self.execution = execution
        self.problem = problem
        self.pheromone = np.ones((len(data), len(data)))  # Matriz de feromonas
        self.best_routes = []

    def run(self):
        start_time = time.time()
        best_cost = math.inf
        best_solutions = []
        for iteration in range(self.n_iterations):
            all_routes = self.construct_solutions()
            self.update_pheromone(all_routes)
            iteration_best = min(all_routes, key=lambda x: x[1])
            if not self.best_routes or iteration_best[1] < self.best_routes[1]:
                self.best_routes = iteration_best
                best_cost = self.best_routes[1]
                best_solutions.append((iteration, iteration_best))
            if self.best_routes[1] < best_cost:
              best_solutions.append((iteration, iteration_best))
              best_cost = self.best_routes[1]
        for idx, (it_idx, it) in enumerate([best_solutions[-1]]):
              print(f"Iteration {it_idx}")
              print(f"Total score = {round(it[1], 2)}")
              print(f"Total distance = {round(it[2],2)}")
              print(f"Waiting/delay cost = {round(it[3], 2)}")
              print(f"Vehicles cost = {it[4] * VEHICLES_COST}")
              print(f"Unattended Cost:{it[5] * UNATTENDED_CUSTOMER_COST} ")
              end_time = time.time()
              elapsed_time = end_time - start_time
              print(f"Execution time for iteration {it_idx}: {elapsed_time:.2f} seconds")
              self.print_routes(it[0])
              #plot_routes(it[0], self.data, it_idx)
              create_routes_table(it[0], self.data, self.problem, self.execution)  # Mostrar tablas detalladas
              return round(it[1], 2), round(it[2], 2), round(it[3], 2), it[4] * VEHICLES_COST, it[5] * UNATTENDED_CUSTOMER_COST, elapsed_time


    def construct_solutions(self):
        all_routes = []
        for _ in range(self.n_ants):
            routes = [[] for _ in range(self.n_vehicles)]  # Una ruta por cada vehículo
            unvisited = set(range(1, len(self.data)))

            for vehicle in range(self.n_vehicles):
                route = [0]  # Empezamos en el almacén (0,0)
                current_time = 0

                while unvisited:
                    current_customer = route[-1]
                    feasible_customers = []

                    for customer in unvisited:
                        travel_time = distance(self.data.loc[current_customer, 'X'], self.data.loc[current_customer, 'Y'],
                                               self.data.loc[customer, 'X'], self.data.loc[customer, 'Y'])
                        arrival_time = current_time + travel_time
                        if arrival_time <= self.data.loc[customer, 'DUE_TIME']:
                            feasible_customers.append(customer)

                    if not feasible_customers:
                        break  # No hay clientes a los que atender


                    probabilities = []
                    for customer in feasible_customers:
                        pheromone = self.pheromone[current_customer][customer] ** self.alpha
                        d = distance(self.data.loc[current_customer, 'X'], self.data.loc[current_customer, 'Y'],
                                                   self.data.loc[customer, 'X'], self.data.loc[customer, 'Y'])
                        if d == 0:
                          probabilities.append(0)
                          continue

                        visibility = (1 / d) ** self.beta
                        probabilities.append(pheromone * visibility)

                    probabilities = np.array(probabilities)
                    probabilities /= probabilities.sum()

                    chosen_customer = np.random.choice(feasible_customers, p=probabilities)
                    route.append(chosen_customer)
                    current_time += distance(self.data.loc[route[-2], 'X'], self.data.loc[route[-2], 'Y'],
                                             self.data.loc[route[-1], 'X'], self.data.loc[route[-1], 'Y'])
                    current_time = max(current_time, self.data.loc[chosen_customer, 'READY_TIME'])
                    current_time += self.data.loc[chosen_customer, 'SERVICE_TIME']
                    unvisited.remove(chosen_customer)

                route.append(0)  # Regreso al almacén
                routes[vehicle] = route

            route_cost, route_distance, wait_delay, vehicles_used, unattended_customers = calculate_routes_cost(routes, self.data)
            all_routes.append((routes, route_cost, route_distance, wait_delay, vehicles_used, unattended_customers))

        return all_routes

    def update_pheromone(self, all_routes):
        self.pheromone *= (1 - self.rho)  # Evaporación de feromonas

        for routes, cost, distance, wait, vehicles, unattended in sorted(all_routes, key=lambda x: x[1])[:self.n_best]:
            for route in routes:
                for i in range(len(route) - 1):
                    self.pheromone[route[i]][route[i+1]] += self.q / cost

    def print_routes(self, routes):
        for i, route in enumerate(routes):
            if len(route) > 2:
              print(f"Vehicle {i+1} route: {' -> '.join(map(str, route))}")

# Parámetros del algoritmo
n_ants = 50  # Número de hormigas
n_best = 10  # Número de mejores rutas para actualizar feromonas
n_iterations = 100  # Número de iteraciones
alpha = 1.0  # Importancia de las feromonas
beta = 2.0  # Importancia de la heurística (inverso de la distancia)
rho = 0.05  # Tasa de evaporación de feromonas
q = 1000  # Cantidad de feromonas añadida por las mejores rutas



NUM_EXECUTIONS = 30
PROBLEM_LIST = [
    'vrptw_10_customers_problem_1.csv',
    'vrptw_10_customers_problem_3.csv',
    'vrptw_10_customers_problem_4.csv',
    'vrptw_10_customers_problem_5.csv',
    'vrptw_10_customers_problem_8.csv',
    'vrptw_10_customers_problem_9.csv',
    'vrptw_20_customers_problem_1.csv',
    'vrptw_20_customers_problem_3.csv',
    'vrptw_20_customers_problem_5.csv',
    'vrptw_20_customers_problem_8.csv',
    'vrptw_20_customers_problem_9.csv',
    'vrptw_20_customers_problem_10.csv',
]
PROBLEM = 11
PROBLEM_LIST = [PROBLEM_LIST[PROBLEM]]

if __name__ == '__main__':
    # Ejecución de los problemas
    for problem in PROBLEM_LIST:
        results = []
        N_VEHICLES = 5 if "_10_" in problem else 10
        print(f"Running problem {problem}")
        for i in range(1, NUM_EXECUTIONS + 1):
            problem_start_time = time.time()
            # Cargar los datos
            data = load_data(f"./{problem}")
            # Ejecución del ACO
            aco = ACO_VRPTW(data, n_ants, n_best, n_iterations, alpha, beta, rho, q, N_VEHICLES, problem, i)
            cost, distance_cost, wait_delay_cost, vehicles_cost, unattended_cost, elapsed_time = aco.run()
            print(f"Execution {i} - time: {elapsed_time:.2f} seconds")
            print("\n")
            results.append((i, problem, cost, distance_cost, wait_delay_cost, vehicles_cost, unattended_cost, elapsed_time, N_VEHICLES))

        COLUMNS = ['Execution', 'Problem', 'Cost', 'Distamce', 'Wait_Delay_Cost', 'Vehicles_Cost', 'Unattended_Cost', 'Time', 'N_Vehicles']
        df = pd.DataFrame(results, columns = COLUMNS)
        df.to_csv(f'./ACO_results_{problem}', index=False)

